In [122]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [123]:
import torch
from sbi.inference import prepare_for_sbi, SNPE
from sbi.simulators.linear_gaussian import diagonal_linear_gaussian
import sbi.utils as sbi_utils
import numpy as np

### Load theta and x

In [124]:
data = np.load('../../../arco_sims/cleaned_up_data.npz', allow_pickle=True)

x = torch.as_tensor(data['data'], dtype=torch.float32)
theta = torch.as_tensor(data['params'], dtype=torch.float32)

### Load prior min and max and gt

In [125]:
import sys
sys.path.append('../')

In [126]:
from parameter_setup import load_ground_truth_params, load_prior_min, load_prior_max

### Load values to standardize x_o

In [127]:
#standardize_vals = np.load('../results/standardize_vals.npz')
#x_mean = standardize_vals['data_mean']
#x_std = standardize_vals['data_std']
#
#theta_mean = standardize_vals['theta_mean']
#theta_std = standardize_vals['theta_std']

x_mean = x.mean(dim=0)
x_std = x.std(dim=0)

theta_mean = theta.mean(dim=0)
theta_std = theta.std(dim=0)

In [128]:
x = (x-x_mean) / x_std
theta = (theta-theta_mean) / theta_std

### Load x_o (which I got from running the simulator with Arco's ground truth params)

In [129]:
import numpy as np
x_o = torch.as_tensor(np.load('../results/observation/x_o.npz')['x_o'], dtype=torch.float32)

x_o = (x_o - x_mean) / x_std
x_o = x_o.unsqueeze(0)

### Create dummy simulator and dummy prior

In [130]:
def dummy_simulator(theta):
    return torch.ones(1,35)

dummy_prior = sbi_utils.BoxUniform(torch.as_tensor(load_prior_min()), torch.as_tensor(load_prior_max()))
_bound = torch.sqrt(torch.as_tensor(3.))
dummy_prior_norm = sbi_utils.BoxUniform(-_bound*torch.ones(35), _bound*torch.ones(35))

In [131]:
simulator, prior, x_shape = prepare_for_sbi(dummy_simulator, dummy_prior_norm)

In [132]:
from bflows.utils.neural_net.get_bounded_flows import get_bflow

boxFlow = get_bflow("nsf_bounded", prior=dummy_prior_norm, context=x_o)

In [133]:
num_datapoints = 10000
inference = SNPE(
    simulator,
    prior, 
    x_shape,
    density_estimator=boxFlow,
    external_data=(theta[:num_datapoints], x[:num_datapoints]),
)

In [134]:
posterior = inference(
    num_rounds=1,
    num_simulations_per_round=0,
    max_num_epochs=5,
    z_score_x=False
)

### Analyse posterior

In [12]:
posterior = posterior.set_default_x(x=x_o)

In [13]:
samples = posterior.sample((10000,), show_progress_bars=True)

# denormalize the samples
samples = samples * theta_std + theta_mean

# convert to list for pickling
samples_list = samples.numpy().tolist()

In [29]:
import pickle
with open('../results/posteriors/200616_dummyPosteriorSNL.pickle', 'wb') as handle:
    pickle.dump(posterior, handle)

In [36]:
with open('../results/samples/200616_dummyPosteriorSNL_samples.pickle', 'wb') as handle:
    pickle.dump(samples_list, handle, protocol=2)

### Posterior predictives
Has to be done in a different virtual env with python 2.